# Entraînement des Modèles de Détection de Fraudes

Ce notebook implémente l'entraînement des modèles Random Forest et XGBoost avec optimisation des hyperparamètres.

## Objectifs

- Charger et préparer les données avec features engineering
- Implémenter et entraîner Random Forest et XGBoost
- Optimiser les hyperparamètres avec validation croisée
- Gérer le déséquilibre des classes avec SMOTE
- Comparer les performances des modèles
- Sauvegarder le meilleur modèle

In [ ]:
# Configuration et imports
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Ajout du chemin racine au sys.path
ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

# Imports de base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json

# Imports ML
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Imports d'optimisation
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

# Imports locaux
from src.data.data_loader import DataLoader
from src.data.preprocessor import Preprocessor
from src.data.feature_engineer import FeatureEngineer
from src.models.random_forest_model import RandomForestModel
from src.models.xgboost_model import XGBoostModel
from src.utils.metrics import calculate_metrics, pr_auc_score

# Configuration des graphiques
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Environnement configuré avec succès")

## 1. Chargement des Données

In [ ]:
# Chargement des données avec features engineering
print("🔄 Chargement des données...")

# Option 1: Utiliser les données avec features engineering du notebook précédent
features_path = "../data/processed/features_engineered.csv"
if os.path.exists(features_path):
    print("📂 Chargement du dataset avec features engineering...")
    df = pd.read_csv(features_path)
else:
    print("📂 Features engineering non trouvé, chargement des données brutes...")
    data_loader = DataLoader()
    df = data_loader.load_data(file_path="../data/raw/creditcard.csv")
    
    # Application du feature engineering
    feature_engineer = FeatureEngineer()
    df = feature_engineer.fit_transform(df)

print(f"📊 Données chargées : {df.shape}")
print(f"📈 Distribution des classes :\n{df['Class'].value_counts(normalize=True)}")

In [ ]:
# Préparation des données
print("🔄 Préparation des données...")

# Séparation features/cible
X = df.drop('Class', axis=1)
y = df['Class']

# Premier split : train/validation/test
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

# Second split : train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp
)

print(f"📊 Train : {X_train.shape}, Validation : {X_val.shape}, Test : {X_test.shape}")
print(f"📈 Ratio classes train : {y_train.value_counts(normalize=True).to_dict()}")
print(f"📈 Ratio classes validation : {y_val.value_counts(normalize=True).to_dict()}")
print(f"📈 Ratio classes test : {y_test.value_counts(normalize=True).to_dict()}")

## 2. Gestion du Déséquilibre des Classes

In [ ]:
# Application de SMOTE pour équilibrer les classes
print("🔄 Application de SMOTE...")

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"📊 Données avant SMOTE : {X_train.shape}")
print(f"📊 Données après SMOTE : {X_train_balanced.shape}")
print(f"📈 Distribution après SMOTE : {y_train_balanced.value_counts(normalize=True).to_dict()}")

# Visualisation de l'équilibrage
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Avant SMOTE
y_train.value_counts().plot(kind='bar', ax=ax1, color=['skyblue', 'salmon'])
ax1.set_title('Distribution des Classes - Avant SMOTE')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Nombre d\'échantillons')
ax1.set_xticklabels(['Légitime (0)', 'Frauduleuse (1)'], rotation=0)

# Après SMOTE
y_train_balanced.value_counts().plot(kind='bar', ax=ax2, color=['skyblue', 'salmon'])
ax2.set_title('Distribution des Classes - Après SMOTE')
ax2.set_xlabel('Classe')
ax2.set_ylabel('Nombre d\'échantillons')
ax2.set_xticklabels(['Légitime (0)', 'Frauduleuse (1)'], rotation=0)

plt.tight_layout()
plt.show()

## 3. Entraînement de Base des Modèles

In [ ]:
# Fonction d'évaluation des modèles
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    """Évalue un modèle et retourne les métriques"""
    print(f"\n🔄 Évaluation de {model_name}...")
    
    # Entraînement
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_proba_train = model.predict_proba(X_train)[:, 1]
    y_proba_val = model.predict_proba(X_val)[:, 1]
    
    # Métriques
    metrics_train = calculate_metrics(y_train, y_pred_train, y_proba_train.reshape(-1, 1))
    metrics_val = calculate_metrics(y_val, y_pred_val, y_proba_val.reshape(-1, 1))
    
    print(f"📊 {model_name} - Train PR-AUC: {metrics_train['pr_auc']:.4f}")
    print(f"📊 {model_name} - Validation PR-AUC: {metrics_val['pr_auc']:.4f}")
    
    return {
        'model': model,
        'metrics_train': metrics_train,
        'metrics_val': metrics_val,
        'y_pred_val': y_pred_val,
        'y_proba_val': y_proba_val
    }

In [ ]:
# Entraînement des modèles de base
print("🔄 Entraînement des modèles de base...")

# Random Forest
rf_model = RandomForestModel({
    'n_estimators': 100,
    'max_depth': 10,
    'random_state': 42,
    'n_jobs': -1
})

rf_results = evaluate_model(rf_model, X_train_balanced, y_train_balanced, X_val, y_val, "Random Forest")

# XGBoost
xgb_model = XGBoostModel({
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'random_state': 42
})

xgb_results = evaluate_model(xgb_model, X_train_balanced, y_train_balanced, X_val, y_val, "XGBoost")

In [ ]:
# Comparaison des performances de base
models_comparison = pd.DataFrame({
    'Modèle': ['Random Forest', 'XGBoost'],
    'PR-AUC Train': [rf_results['metrics_train']['pr_auc'], xgb_results['metrics_train']['pr_auc']],
    'PR-AUC Validation': [rf_results['metrics_val']['pr_auc'], xgb_results['metrics_val']['pr_auc']],
    'Precision': [rf_results['metrics_val']['precision'], xgb_results['metrics_val']['precision']],
    'Recall': [rf_results['metrics_val']['recall'], xgb_results['metrics_val']['recall']],
    'F1-Score': [rf_results['metrics_val']['f1'], xgb_results['metrics_val']['f1']]
})

print("📊 Comparaison des performances de base :")
display(models_comparison.round(4))

## 4. Optimisation des Hyperparamètres

In [ ]:
# Fonctions d'optimisation Optuna
def objective_rf(trial):
    """Fonction objectif pour Random Forest"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = RandomForestModel(params)
    model.fit(X_train_balanced, y_train_balanced)
    
    y_proba = model.predict_proba(X_val)[:, 1]
    pr_auc = pr_auc_score(y_val, y_proba)
    
    return pr_auc

def objective_xgb(trial):
    """Fonction objectif pour XGBoost"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42
    }
    
    model = XGBoostModel(params)
    model.fit(X_train_balanced, y_train_balanced)
    
    y_proba = model.predict_proba(X_val)[:, 1]
    pr_auc = pr_auc_score(y_val, y_proba)
    
    return pr_auc

In [ ]:
# Optimisation Random Forest
print("🔄 Optimisation des hyperparamètres Random Forest...")

study_rf = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    pruner=MedianPruner()
)

study_rf.optimize(objective_rf, n_trials=20, timeout=300)

print("✅ Optimisation RF terminée")
print(".4f")
print(f"📊 Meilleurs paramètres RF : {study_rf.best_params}")

In [ ]:
# Optimisation XGBoost
print("🔄 Optimisation des hyperparamètres XGBoost...")

study_xgb = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42),
    pruner=MedianPruner()
)

study_xgb.optimize(objective_xgb, n_trials=20, timeout=300)

print("✅ Optimisation XGB terminée")
print(".4f")
print(f"📊 Meilleurs paramètres XGB : {study_xgb.best_params}")

In [ ]:
# Entraînement des modèles optimisés
print("🔄 Entraînement des modèles optimisés...")

# Random Forest optimisé
rf_optimized = RandomForestModel(study_rf.best_params)
rf_opt_results = evaluate_model(rf_optimized, X_train_balanced, y_train_balanced, X_val, y_val, "Random Forest Optimisé")

# XGBoost optimisé
xgb_optimized = XGBoostModel(study_xgb.best_params)
xgb_opt_results = evaluate_model(xgb_optimized, X_train_balanced, y_train_balanced, X_val, y_val, "XGBoost Optimisé")

In [ ]:
# Comparaison finale
final_comparison = pd.DataFrame({
    'Modèle': ['RF Base', 'RF Optimisé', 'XGB Base', 'XGB Optimisé'],
    'PR-AUC Validation': [
        rf_results['metrics_val']['pr_auc'],
        rf_opt_results['metrics_val']['pr_auc'],
        xgb_results['metrics_val']['pr_auc'],
        xgb_opt_results['metrics_val']['pr_auc']
    ],
    'Precision': [
        rf_results['metrics_val']['precision'],
        rf_opt_results['metrics_val']['precision'],
        xgb_results['metrics_val']['precision'],
        xgb_opt_results['metrics_val']['precision']
    ],
    'Recall': [
        rf_results['metrics_val']['recall'],
        rf_opt_results['metrics_val']['recall'],
        xgb_results['metrics_val']['recall'],
        xgb_opt_results['metrics_val']['recall']
    ],
    'F1-Score': [
        rf_results['metrics_val']['f1'],
        rf_opt_results['metrics_val']['f1'],
        xgb_results['metrics_val']['f1'],
        xgb_opt_results['metrics_val']['f1']
    ]
})

print("📊 Comparaison finale des modèles :")
display(final_comparison.round(4))

# Sélection du meilleur modèle
best_model_idx = final_comparison['PR-AUC Validation'].idxmax()
best_model_name = final_comparison.loc[best_model_idx, 'Modèle']
best_pr_auc = final_comparison.loc[best_model_idx, 'PR-AUC Validation']

print(f"\n🏆 MEILLEUR MODÈLE : {best_model_name}")
print(".4f")

## 5. Validation Croisée Finale

In [ ]:
# Validation croisée sur le meilleur modèle
print("🔄 Validation croisée finale...")

# Combinaison des données d'entraînement et validation pour CV
X_train_full = pd.concat([X_train_balanced, X_val])
y_train_full = pd.concat([y_train_balanced, y_val])

# Sélection du meilleur modèle
if best_model_name == 'RF Optimisé':
    best_model = rf_optimized
    model_params = study_rf.best_params
elif best_model_name == 'XGB Optimisé':
    best_model = xgb_optimized
    model_params = study_xgb.best_params
else:
    best_model = rf_optimized  # Par défaut
    model_params = study_rf.best_params

# Validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(
    best_model.model, X_train_full, y_train_full, 
    cv=cv, scoring='average_precision', n_jobs=-1
)

print("📊 Scores de validation croisée (PR-AUC) :")
for i, score in enumerate(cv_scores, 1):
    print(".4f")
print(".4f")
print(".4f")

## 6. Évaluation Finale sur le Test Set

In [ ]:
# Évaluation finale sur les données de test
print("🔄 Évaluation finale sur le test set...")

# Entraînement sur toutes les données d'entraînement
best_model.fit(X_train_full, y_train_full)

# Prédictions sur le test set
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)[:, 1]

# Métriques finales
final_metrics = calculate_metrics(y_test, y_pred_test, y_proba_test.reshape(-1, 1))

print("📊 MÉTRIQUES FINALES SUR LE TEST SET :")
print(f"PR-AUC : {final_metrics['pr_auc']:.4f}")
print(f"ROC-AUC : {final_metrics['roc_auc']:.4f}")
print(f"Precision : {final_metrics['precision']:.4f}")
print(f"Recall : {final_metrics['recall']:.4f}")
print(f"F1-Score : {final_metrics['f1']:.4f}")
print(f"MCC : {final_metrics['mcc']:.4f}")

# Rapport de classification détaillé
print("\n📋 RAPPORT DE CLASSIFICATION DÉTAILLÉ :")
print(classification_report(y_test, y_pred_test, target_names=['Légitime', 'Frauduleuse']))

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, y_pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Légitime', 'Frauduleuse'],
            yticklabels=['Légitime', 'Frauduleuse'])
plt.title('Matrice de Confusion - Test Set')
plt.xlabel('Prédiction')
plt.ylabel('Réalité')
plt.tight_layout()
plt.show()

In [ ]:
# Courbe ROC et Precision-Recall
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_proba_test)
roc_auc = auc(fpr, tpr)

ax1.plot(fpr, tpr, color='darkorange', lw=2, label='.2f')
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('Taux de Faux Positifs')
ax1.set_ylabel('Taux de Vrais Positifs')
ax1.set_title('Courbe ROC')
ax1.legend(loc="lower right")
ax1.grid(True)

# Courbe Precision-Recall
from sklearn.metrics import precision_recall_curve
precision, recall, _ = precision_recall_curve(y_test, y_proba_test)
pr_auc = auc(recall, precision)

ax2.plot(recall, precision, color='blue', lw=2, label='.2f')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Rappel')
ax2.set_ylabel('Précision')
ax2.set_title('Courbe Precision-Recall')
ax2.legend(loc="lower left")
ax2.grid(True)

plt.tight_layout()
plt.show()

## 7. Sauvegarde du Modèle

In [ ]:
# Sauvegarde du modèle et des métadonnées
print("💾 Sauvegarde du modèle final...")

import joblib
import os

# Création des dossiers
os.makedirs("../models/trained", exist_ok=True)
os.makedirs("../models/metadata", exist_ok=True)

# Sauvegarde du modèle
model_path = "../models/trained/best_model.pkl"
joblib.dump(best_model, model_path)
print(f"✅ Modèle sauvegardé : {model_path}")

# Sauvegarde des métadonnées
metadata = {
    'model_name': best_model_name,
    'model_type': type(best_model).__name__,
    'best_params': model_params,
    'final_metrics': final_metrics,
    'cv_scores': cv_scores.tolist(),
    'cv_mean': cv_scores.mean(),
    'cv_std': cv_scores.std(),
    'training_date': datetime.now().isoformat(),
    'data_shape': X_train_full.shape,
    'feature_names': X_train_full.columns.tolist(),
    'optuna_study': {
        'rf_best_score': study_rf.best_value,
        'xgb_best_score': study_xgb.best_value,
        'rf_params': study_rf.best_params,
        'xgb_params': study_xgb.best_params
    }
}

metadata_path = "../models/metadata/model_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2, default=str)

print(f"✅ Métadonnées sauvegardées : {metadata_path}")

## 8. Résumé et Conclusions

In [ ]:
print("🎯 RÉSUMÉ DE L'ENTRAÎNEMENT DES MODÈLES")
print("=" * 60)

print(f"🏆 MEILLEUR MODÈLE : {best_model_name}")
print(f"📊 PR-AUC Validation Croisée : {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"📊 PR-AUC Test Final : {final_metrics['pr_auc']:.4f}")
print(f"📊 ROC-AUC Test Final : {final_metrics['roc_auc']:.4f}")

print("\n🔧 TECHNIQUES UTILISÉES :")
print("• Feature Engineering avancé")
print("• Gestion du déséquilibre avec SMOTE")
print("• Optimisation des hyperparamètres avec Optuna")
print("• Validation croisée stratifiée")
print("• Métriques adaptées aux classes déséquilibrées")

print("\n📈 AMÉLIORATIONS APPORTÉES :")
print(".4f")
print(".4f")
print(".4f")

print("\n💾 FICHIERS SAUVEGARDÉS :")
print(f"• Modèle : {model_path}")
print(f"• Métadonnées : {metadata_path}")
print("• Features Engineer : ../models/feature_engineer.pkl")

print("\n🚀 MODÈLE PRÊT POUR LA PRODUCTION !")